# Nine Test

This breaks the continuity that we have been following. But it's important because for debug properly we need a proper batch. The eight test continues being the important one.

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Clone

In [ ]:
%cd XHUMAN

In [ ]:
!uv pip install -e .[pi]

In [ ]:
import torch
from xhuman.configs.default import LerobotDatasetConfig

dataset_config = LerobotDatasetConfig(
    repo_id="NONHUMAN_RESEARCH/TEST-RECORD-ANNOTATIONS",  
)

device = torch.device(policy_config.device if policy_config.device else "cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from xhuman.configs.train import TrainPipelineConfigXHUMAN

cfg = TrainPipelineConfigXHUMAN(
    dataset=dataset_config,
    policy=policy_config,
    batch_size=50,
    num_workers=4,
    split_ratio=0.8,
)

cfg.validate()

In [ ]:
from xhuman.policies.pi05.configuration_pi05 import PI05Config

policy_config = PI05Config(
    device=device,
)

In [ ]:
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs

ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
accelerator = Accelerator(step_scheduler_with_optimizer=False, kwargs_handlers=[ddp_kwargs])

In [ ]:
dataset = make_dataset_xhuman(cfg)
episodes = list(range(dataset.meta.total_episodes))
train_episodes, _ = split_train_eval_episodes(episodes, split_ratio=cfg.split_ratio, seed=42)
if 30 in train_episodes:
    train_episodes.remove(30)

# Eliminar dataset para liberar memoria (igual que en train_val_pi05.py)
del dataset

In [ ]:
from xhuman.policies.factory import make_xhuman_policy

policy = make_xhuman_policy(cfg=cfg.policy, ds_meta=dataset.meta)

processor_kwargs = {"dataset_stats": dataset.meta.stats}
if cfg.policy.pretrained_path is not None:
    processor_kwargs["preprocessor_overrides"] = {
        "device_processor": {"device": device.type},
        "normalizer_processor": {
            "stats": dataset.meta.stats,
            "features": {**policy.config.input_features, **policy.config.output_features},
            "norm_map": policy.config.normalization_mapping,
        },
    }


In [ ]:
from xhuman.policies.pi05.processor_pi05 import (
    make_pi05_pre_post_processors_ki,
)

preprocessor, _ = make_pi05_pre_post_processors_ki(
    policy_cfg=cfg.policy,
    pretrained_path=cfg.policy.pretrained_path,
    **processor_kwargs,
)

In [ ]:
from xhuman.datasets.factory import make_dataset_xhuman

cfg.dataset.episodes = train_episodes
dataset = make_dataset_xhuman(cfg)

In [ ]:
from lerobot.datasets.sampler import EpisodeAwareSampler

# Crear dataloader
if hasattr(cfg.policy, "drop_n_last_frames"):
    sampler = EpisodeAwareSampler(
        dataset.meta.episodes["dataset_from_index"],
        dataset.meta.episodes["dataset_to_index"],
        drop_n_last_frames=cfg.policy.drop_n_last_frames,
        shuffle=True,
    )
else:
    sampler = None


In [ ]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=cfg.num_workers,
    batch_size=cfg.batch_size,
    shuffle=(sampler is None) and not cfg.dataset.streaming,
    sampler=sampler,
    pin_memory=device.type == "cuda",
    drop_last=False,
    prefetch_factor=2 if cfg.num_workers > 0 else None,
)

In [ ]:
import cycle

In [ ]:
dataloader = accelerator.prepare(dataloader)
dl_iter = cycle(dataloader)

In [ ]:
# Let's inspect the raw batch BEFORE preprocessing to see the original structure
print("=" * 80)
print("RAW BATCH (before preprocessor):")
print("=" * 80)

# Get a fresh batch before preprocessing
raw_batch = next(dl_iter)
print(f"\nRaw batch keys: {list(raw_batch.keys())}\n")

for key, value in raw_batch.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape={value.shape}, dtype={value.dtype}")
    elif isinstance(value, (list, tuple)):
        print(f"  {key}: {type(value).__name__} of length {len(value)}")
        if len(value) > 0 and isinstance(value[0], torch.Tensor):
            print(f"    First element shape: {value[0].shape}")
    else:
        print(f"  {key}: {type(value).__name__} = {value}")

print("\n" + "=" * 80)
print("COMPARISON:")
print("=" * 80)
print("Raw batch has the original dataset structure")
print("Processed batch (after preprocessor) has the structure expected by the policy")

## Understanding `cfg.steps` vs Epochs

In `train_val_pi05.py`, the training loop uses:

```python
for _ in range(step, cfg.steps):
    batch = next(dl_iter)  # Get next batch from infinite iterator
    batch = preprocessor(batch)
    # ... training step ...
    step += 1
```

**Key points:**

1. **`cfg.steps`** = Total number of training steps (gradient updates), NOT epochs
2. **`cycle(dataloader)`** creates an infinite iterator that automatically restarts when exhausted
3. This approach is common in RL/robotics because:
   - You care more about number of gradient updates than epochs
   - Datasets can be very large, so epochs might take too long
   - You can easily control training duration with a step count

**Example calculation:**
- Dataset: 10,000 samples
- Batch size: 50
- Batches per epoch: 10,000 / 50 = 200 batches
- If `cfg.steps = 1000`:
  - We do 1000 gradient updates
  - This equals 1000 / 200 = 5 epochs worth of data
  - But we don't explicitly count epochs, just steps

In [ ]:
# Let's inspect the raw batch BEFORE preprocessing to see the original structure
print("=" * 80)
print("RAW BATCH (before preprocessor):")
print("=" * 80)

# Get a fresh batch before preprocessing
raw_batch = next(dl_iter)
print(f"\nRaw batch keys: {list(raw_batch.keys())}\n")

for key, value in raw_batch.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape={value.shape}, dtype={value.dtype}")
    elif isinstance(value, (list, tuple)):
        print(f"  {key}: {type(value).__name__} of length {len(value)}")
        if len(value) > 0 and isinstance(value[0], torch.Tensor):
            print(f"    First element shape: {value[0].shape}")
    else:
        print(f"  {key}: {type(value).__name__} = {value}")

print("\n" + "=" * 80)
print("COMPARISON:")
print("=" * 80)
print("Raw batch has the original dataset structure")
print("Processed batch (after preprocessor) has the structure expected by the policy")

In [ ]:
# Obtener batch procesado (igual que next(dl_iter) en train_val_pi05.py)
batch = next(dl_iter)
batch = preprocessor(batch)

In [ ]:
# El batch está listo para usar
print("Batch keys:", list(batch.keys()))
for key, value in batch.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape={value.shape}, dtype={value.dtype}")